In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from lightgbm import LGBMRegressor
from sklearn.cluster import KMeans
import warnings
from sklearn.feature_selection import VarianceThreshold
from scipy import stats

# Suppress warnings for cleaner output
warnings.filterwarnings("ignore")

In [2]:
# =============================================================================
# DATA LOADING AND INITIAL SETUP
# =============================================================================

print("Loading data...")
# Load development (training) and evaluation (test) datasets
df_dev = pd.read_csv("/content/development.csv")    # Training data with price labels
df_eval = pd.read_csv("/content/evaluation.csv")    # Test data without prices to predict

print(f"Training data shape: {df_dev.shape}")
print(f"Test data shape: {df_eval.shape}")
print(f"Training data columns: {list(df_dev.columns)}")
print(f"Test data columns: {list(df_eval.columns)}")

# Store original test IDs for final submission
test_ids_for_submission = df_eval["id"].copy()

# Prepare datasets for concatenation
df_eval["price"] = np.nan  # Add empty price column to test set
df_dev['is_dev'] = 1       # Flag for development/training data
df_eval['is_dev'] = 0      # Flag for evaluation/test data

# Handle missing category column in test set if needed
if "category" not in df_eval.columns:
    df_eval["category"] = np.nan

# Combine training and test datasets for unified preprocessing
df = pd.concat([df_dev, df_eval], ignore_index=True, sort=False)
print(f"Combined dataset shape: {df.shape}")
print(f"Training samples: {(df['is_dev'] == 1).sum()}")
print(f"Test samples: {(df['is_dev'] == 0).sum()}")


Loading data...
Training data shape: (79589, 21)
Test data shape: (19898, 20)
Training data columns: ['category', 'title', 'body', 'amenities', 'bathrooms', 'bedrooms', 'currency', 'fee', 'has_photo', 'pets_allowed', 'price', 'price_type', 'square_feet', 'address', 'cityname', 'state', 'latitude', 'longitude', 'source', 'time', 'id']
Test data columns: ['category', 'title', 'body', 'amenities', 'bathrooms', 'bedrooms', 'currency', 'fee', 'has_photo', 'pets_allowed', 'price_type', 'square_feet', 'address', 'cityname', 'state', 'latitude', 'longitude', 'source', 'time', 'id']
Combined dataset shape: (99487, 22)
Training samples: 79589
Test samples: 19898


In [3]:
# =============================================================================
# TARGET ENGINEERING: LOG TRANSFORMATION
# =============================================================================

print("Target engineering...")
# Store original target values for analysis
y_train_original = df[df['is_dev'] == 1]["price"].copy()
# Apply log transformation to normalize price distribution
df.loc[df['is_dev'] == 1, 'price_log'] = np.log1p(df[df['is_dev'] == 1]["price"])

# Compare skewness before and after transformation
original_skew = stats.skew(y_train_original)
log_skew = stats.skew(df[df['is_dev'] == 1]['price_log'])
print(f"Target skewness - Original: {original_skew:.3f}, Log-transformed: {log_skew:.3f}")


Target engineering...
Target skewness - Original: 9.129, Log-transformed: 0.436


In [4]:
# =============================================================================
# CONSERVATIVE OUTLIER DETECTION AND REMOVAL
# =============================================================================

print("Conservative outlier removal...")
# Use conservative approach to maintain more training data
train_mask = df['is_dev'] == 1
Q1 = df.loc[train_mask, 'price'].quantile(0.15)  # More conservative than Q1 (0.25)
Q3 = df.loc[train_mask, 'price'].quantile(0.85)  # More conservative than Q3 (0.75)
IQR = Q3 - Q1

# Wider bounds to remove only extreme outliers
lower_bound = Q1 - 2.5 * IQR  # Standard is 1.5, using 2.5 for conservation
upper_bound = Q3 + 2.5 * IQR

# Identify extreme outliers
outlier_mask = train_mask & ((df['price'] < lower_bound) | (df['price'] > upper_bound))
outliers_removed = outlier_mask.sum()
total_training = train_mask.sum()
print(f"Outliers removed: {outliers_removed} out of {total_training} ({outliers_removed/total_training*100:.1f}%)")

# Remove outliers only if they represent less than 5% of data
if outliers_removed / total_training < 0.05:
    df = df[~outlier_mask].reset_index(drop=True)
    print("Outliers removed successfully")
else:
    print("Too many outliers detected, keeping all data")

# =============================================================================
# ADVANCED DATA CLEANING AND IMPUTATION
# =============================================================================

print("Advanced data cleaning...")

# BATHROOMS: Hierarchical imputation using related features
df["bathrooms"].fillna(
    df.groupby(['bedrooms', 'square_feet'])['bathrooms'].transform('median'),
    inplace=True
)
df["bathrooms"].fillna(1, inplace=True)  # Default fallback

# BEDROOMS: Similar hierarchical approach
df["bedrooms"].fillna(
    df.groupby(['bathrooms', 'square_feet'])['bedrooms'].transform('median'),
    inplace=True
)
df["bedrooms"].fillna(1, inplace=True)

# SQUARE FEET: Multi-level imputation strategy
df["square_feet"].fillna(
    df.groupby(['bedrooms', 'bathrooms'])['square_feet'].transform('median'),
    inplace=True
)
df["square_feet"].fillna(df["square_feet"].median(), inplace=True)

# Conservative clipping for square feet to remove unrealistic values
df["square_feet"] = df["square_feet"].clip(
    lower=200,  # Minimum realistic apartment size
    upper=df["square_feet"].quantile(0.995)  # 99.5th percentile to preserve more data
)

# BINARY FEATURES: Clean encoding
df["fee"] = df["fee"].replace({"No": 0, "Yes": 1}).fillna(0).astype(int)
df["has_photo"] = df["has_photo"].astype(str).str.lower().map({
    "true": 1, "1": 1, "yes": 1,
    "false": 0, "0": 0, "no": 0
}).fillna(0).astype(int)

# CATEGORICAL FEATURES: Fill missing values with 'unknown'
categorical_cols = ["pets_allowed", "category", "cityname", "state", "price_type"]
for col in categorical_cols:
    df[col].fillna("unknown", inplace=True)


Conservative outlier removal...
Outliers removed: 344 out of 79589 (0.4%)
Outliers removed successfully
Advanced data cleaning...


In [5]:
# =============================================================================
# COMPREHENSIVE FEATURE ENGINEERING
# =============================================================================

print("Extended feature engineering...")

# 1. ROOM-BASED FEATURES
# Ratio of bedrooms to bathrooms (with small epsilon to avoid division by zero)
df["rooms_ratio"] = df["bedrooms"] / (df["bathrooms"] + 0.1)
df["rooms_ratio"] = df["rooms_ratio"].clip(upper=10)  # Cap extreme ratios
df["total_rooms"] = df["bedrooms"] + df["bathrooms"]
df["rooms_per_sqft"] = df["total_rooms"] / (df["square_feet"] + 1)

# 2. PRICE PER SQUARE FOOT ESTIMATES (only for training data)
train_data = df[df['is_dev'] == 1]
if len(train_data) > 0:
    # Calculate average price per square foot by city
    city_price_per_sqft = train_data.groupby('cityname')['price'].sum() / train_data.groupby('cityname')['square_feet'].sum()
    df['city_price_per_sqft'] = df['cityname'].map(city_price_per_sqft).fillna(city_price_per_sqft.median())

    # Estimate price based on city's price per square foot
    df['estimated_price_by_city'] = df['square_feet'] * df['city_price_per_sqft']

# 3. TEMPORAL FEATURES
df['time'] = pd.to_datetime(df['time'], errors='coerce')
df['listing_month'] = df['time'].dt.month.fillna(0).astype(int)
df['listing_day_of_week'] = df['time'].dt.dayofweek.fillna(0).astype(int)
df['is_weekend'] = (df['listing_day_of_week'] >= 5).astype(int)
df['listing_hour'] = df['time'].dt.hour.fillna(12).astype(int)
df['is_business_hours'] = ((df['listing_hour'] >= 9) & (df['listing_hour'] <= 17)).astype(int)

# 4. GEOGRAPHIC FEATURES
df['latitude'].fillna(df['latitude'].median(), inplace=True)
df['longitude'].fillna(df['longitude'].median(), inplace=True)

# Geographic clustering to capture location-based patterns
if len(df[df['latitude'].notna() & df['longitude'].notna()]) > 100:
    coords = df[['latitude', 'longitude']].fillna(0)
    kmeans = KMeans(n_clusters=20, random_state=42, n_init=10)
    df['geo_cluster'] = kmeans.fit_predict(coords)
else:
    df['geo_cluster'] = 0

# 5. POLYNOMIAL AND INTERACTION FEATURES
# Create polynomial features to capture non-linear relationships
df['sqft_squared'] = df['square_feet'] ** 2
df['total_rooms_squared'] = df['total_rooms'] ** 2
df['bedrooms_bathrooms_interaction'] = df['bedrooms'] * df['bathrooms']
df['sqft_rooms_interaction'] = df['square_feet'] * df['total_rooms']

# 6. LOG FEATURES to improve distributions
df['log_square_feet'] = np.log1p(df['square_feet'])
df['log_total_rooms'] = np.log1p(df['total_rooms'])



Extended feature engineering...


In [6]:
# =============================================================================
# ADVANCED TF-IDF TEXT PROCESSING
# =============================================================================

def tfidf_features_advanced(df_input, column, max_features, ngram_range=(1,2)):
    """
    Create advanced TF-IDF features from text column.

    Args:
        df_input: Input dataframe
        column: Column name to process
        max_features: Maximum number of features to extract
        ngram_range: Range of n-grams to consider

    Returns:
        DataFrame with TF-IDF features
    """
    print(f"⚙️ Advanced TF-IDF on '{column}' (max_features={max_features})")

    # Clean and preprocess text data
    text_data = df_input[column].fillna("").astype(str)
    text_data = text_data.str.lower().str.replace(r'[^\w\s]', ' ', regex=True)

    # Configure TF-IDF vectorizer with optimized parameters
    vectorizer = TfidfVectorizer(
        max_features=max_features,
        stop_words="english",
        ngram_range=ngram_range,
        min_df=2,                      # Minimum document frequency
        max_df=0.98,                   # Maximum document frequency
        sublinear_tf=True,             # Apply sublinear scaling
        norm='l2'                      # L2 normalization
    )

    tfidf_matrix = vectorizer.fit_transform(text_data)

    return pd.DataFrame(
        tfidf_matrix.toarray(),
        columns=[f"{column}_tfidf_{i}" for i in range(tfidf_matrix.shape[1])],
        index=df_input.index
    )

# Apply TF-IDF to text columns with different configurations
df = pd.concat([df, tfidf_features_advanced(df, "title", 200, ngram_range=(1,3))], axis=1)
df = pd.concat([df, tfidf_features_advanced(df, "body", 400, ngram_range=(1,2))], axis=1)


⚙️ Advanced TF-IDF on 'title' (max_features=200)
⚙️ Advanced TF-IDF on 'body' (max_features=400)


In [7]:
# =============================================================================
# ADVANCED AMENITIES PROCESSING
# =============================================================================

def process_amenities_advanced(amenity_series, min_count=8):
    """
    Process amenities text into binary features with normalization.

    Args:
        amenity_series: Series containing amenity text
        min_count: Minimum count for amenity to be included

    Returns:
        DataFrame with binary amenity features
    """
    print("⚙️ Advanced amenities processing")

    def normalize_amenity(text):
        """Normalize amenity names to standard categories."""
        if pd.isna(text):
            return []

        amenities = [a.strip().lower() for a in str(text).split(",") if a.strip()]

        normalized = []
        for a in amenities:
            # Group similar amenities into standard categories
            if 'parking' in a or 'garage' in a:
                normalized.append('parking')
            elif 'pet' in a or 'dog' in a or 'cat' in a:
                normalized.append('pet_friendly')
            elif 'laundry' in a or 'washer' in a or 'dryer' in a:
                normalized.append('laundry')
            elif 'gym' in a or 'fitness' in a:
                normalized.append('gym')
            elif 'pool' in a:
                normalized.append('pool')
            elif 'balcony' in a or 'patio' in a or 'deck' in a:
                normalized.append('outdoor_space')
            elif 'dishwasher' in a:
                normalized.append('dishwasher')
            elif 'hardwood' in a or 'wood' in a:
                normalized.append('hardwood_floors')
            else:
                normalized.append(a)
        return normalized

    # Process all amenities and count frequencies
    all_amenities = amenity_series.apply(normalize_amenity).explode()
    amenity_counts = all_amenities.value_counts()
    common_amenities = amenity_counts[amenity_counts >= min_count].index.tolist()

    print(f"Common amenities selected: {len(common_amenities)}")

    # Create binary feature matrix
    amenity_data = pd.DataFrame(
        0,
        index=amenity_series.index,
        columns=[f"amenity_{a.replace(' ', '_').replace('-', '_')}"
                for a in sorted(common_amenities)]
    )

    # Fill binary features
    for i, entry in amenity_series.items():
        normalized_amenities = normalize_amenity(entry)
        for a in normalized_amenities:
            col_name = f"amenity_{a.replace(' ', '_').replace('-', '_')}"
            if col_name in amenity_data.columns:
                amenity_data.at[i, col_name] = 1

    # Add total amenities count
    amenity_data['total_amenities'] = amenity_data.sum(axis=1)

    return amenity_data

# Process amenities with relaxed minimum count
df = pd.concat([df, process_amenities_advanced(df["amenities"], min_count=6)], axis=1)

# =============================================================================
# DATA CLEANUP
# =============================================================================

# Remove columns no longer needed for modeling
columns_to_drop = ["title", "body", "amenities", "currency", "address", "source", "time"]
df.drop(columns=[col for col in columns_to_drop if col in df.columns],
        inplace=True, errors="ignore")


⚙️ Advanced amenities processing
Common amenities selected: 26


In [8]:
# =============================================================================
# OPTIMIZED CATEGORICAL ENCODING
# =============================================================================

print("Optimized categorical encoding...")

def target_encode(df, cat_col, target_col, alpha=15):
    """
    Apply target encoding with smoothing to categorical variables.

    Args:
        df: Input dataframe
        cat_col: Categorical column to encode
        target_col: Target column for encoding
        alpha: Smoothing parameter (higher = more smoothing)

    Returns:
        Modified dataframe with target encoded column
    """
    train_mask = df['is_dev'] == 1
    if train_mask.sum() == 0:
        return df

    # Calculate global target mean
    target_mean = df.loc[train_mask, target_col].mean()

    # Calculate category-specific statistics
    category_stats = df.loc[train_mask].groupby(cat_col)[target_col].agg(['mean', 'count'])

    # Apply smoothing formula
    smoothed_means = (
        (category_stats['mean'] * category_stats['count'] + target_mean * alpha) /
        (category_stats['count'] + alpha)
    )

    # Map encoded values
    df[f'{cat_col}_target_encoded'] = df[cat_col].map(smoothed_means).fillna(target_mean)

    return df

# Apply target encoding to high-cardinality categorical features
high_cardinality_cols = ['cityname', 'state', 'geo_cluster']

for col in high_cardinality_cols:
    if col in df.columns:
        df = target_encode(df, col, 'price_log', alpha=25)

# Apply one-hot encoding to low-cardinality categorical features
low_cardinality_cols = [
    "category", "pets_allowed", "price_type",
    "listing_month", "listing_day_of_week", "listing_hour"
]

df = pd.get_dummies(
    df,
    columns=[col for col in low_cardinality_cols if col in df.columns],
    drop_first=True,  # Avoid multicollinearity
    dummy_na=True     # Create dummy for missing values
)

# =============================================================================
# FINAL DATA CLEANING
# =============================================================================

print("Final data cleaning...")

# Convert all object columns to numeric
for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = pd.to_numeric(df[col], errors='coerce')
        df[col].fillna(0, inplace=True)

# Fill any remaining missing values
df.fillna(0, inplace=True)

Optimized categorical encoding...
Final data cleaning...


In [9]:
# =============================================================================
# FEATURE SELECTION
# =============================================================================

print("Feature selection...")

# Prepare training and test data
X_train_full = df[df['is_dev'] == 1].drop(columns=["price", "price_log", "id", "is_dev"])
y_train_full = df[df['is_dev'] == 1]["price_log"]
X_test = df[df['is_dev'] == 0].drop(columns=["price", "price_log", "id", "is_dev"])

print(f"Features before selection: {X_train_full.shape[1]}")

# Remove low-variance features (more permissive threshold)
variance_selector = VarianceThreshold(threshold=0.005)
X_train_full = pd.DataFrame(
    variance_selector.fit_transform(X_train_full),
    columns=X_train_full.columns[variance_selector.get_support()],
    index=X_train_full.index
)
X_test = pd.DataFrame(
    variance_selector.transform(X_test),
    columns=X_train_full.columns,
    index=X_test.index
)

# Select top features based on correlation with target
corr_with_target = X_train_full.corrwith(y_train_full).abs().sort_values(ascending=False)
top_features = corr_with_target.head(800).index.tolist()

X_train_full = X_train_full[top_features]
X_test = X_test[top_features]

print(f"Features after selection: {X_train_full.shape[1]}")

# =============================================================================
# COLUMN NAME CLEANING FOR LIGHTGBM COMPATIBILITY
# =============================================================================

def clean_column_names(df_input):
    """Clean column names to be compatible with LightGBM."""
    cols = df_input.columns
    new_cols = []
    for col in cols:
        # Remove or replace problematic characters
        new_col = col.replace("[", "").replace("]", "").replace("<", "").replace(">", "")
        new_col = new_col.replace("=", "_eq_").replace(":", "_col_").replace("/", "_div_")
        new_col = new_col.replace(" ", "_").replace("-", "_").replace("__", "_")
        new_col = new_col.replace(".", "_dot_").replace(",", "_comma_")
        new_cols.append(new_col)
    df_input.columns = new_cols
    return df_input

X_train_full = clean_column_names(X_train_full)
X_test = clean_column_names(X_test)

Feature selection...
Features before selection: 669
Features after selection: 111


In [10]:
# =============================================================================
# FINAL MODEL TRAINING - NO VALIDATION SPLIT
# =============================================================================

print("Training final model on ALL training data - NO VALIDATION!")

# Optimized parameters for using all training data
final_params = {
    'objective': 'mae',              # Mean Absolute Error objective
    'metric': 'mae',                 # MAE metric for evaluation
    'boosting_type': 'gbdt',         # Gradient Boosting Decision Tree
    'num_leaves': 180,               # Number of leaves in tree
    'learning_rate': 0.04,           # Learning rate
    'feature_fraction': 0.8,         # Fraction of features to use
    'bagging_fraction': 0.7,         # Fraction of data to use for bagging
    'bagging_freq': 5,               # Frequency of bagging
    'min_child_samples': 10,         # Minimum samples in leaf
    'reg_alpha': 0.2,                # L1 regularization
    'reg_lambda': 1.5,               # L2 regularization
    'max_depth': 16,                 # Maximum tree depth
    'n_estimators': 2500,            # Number of boosting rounds
    'random_state': 42,              # Random seed for reproducibility
    'n_jobs': -1,                    # Use all available cores
    'verbosity': -1                  # Suppress verbose output
}

print(f"Training with parameters: {final_params}")
print(f"Training data shape: {X_train_full.shape}")
print(f"Test data shape: {X_test.shape}")
print(f"USING ALL {X_train_full.shape[0]} TRAINING SAMPLES!")

# Train model on all available training data
model = LGBMRegressor(**final_params)
model.fit(X_train_full, y_train_full)

print("Training completed on ALL training data!")

# =============================================================================
# PREDICTION AND POST-PROCESSING
# =============================================================================

print("Making final predictions...")

# Generate predictions on test set
test_preds_log = model.predict(X_test)
y_pred_final = np.expm1(test_preds_log)  # Convert back from log space

print("No internal validation - maximum performance on test set!")

# Intelligent post-processing
print("Intelligent post-processing...")

# Use all training data for clipping bounds
all_train_prices = np.expm1(y_train_full)



Training final model on ALL training data - NO VALIDATION!
Training with parameters: {'objective': 'mae', 'metric': 'mae', 'boosting_type': 'gbdt', 'num_leaves': 180, 'learning_rate': 0.04, 'feature_fraction': 0.8, 'bagging_fraction': 0.7, 'bagging_freq': 5, 'min_child_samples': 10, 'reg_alpha': 0.2, 'reg_lambda': 1.5, 'max_depth': 16, 'n_estimators': 2500, 'random_state': 42, 'n_jobs': -1, 'verbosity': -1}
Training data shape: (79245, 111)
Test data shape: (19898, 111)
USING ALL 79245 TRAINING SAMPLES!
Training completed on ALL training data!
Making final predictions...
No internal validation - maximum performance on test set!
Intelligent post-processing...


In [11]:
# =============================================================================
# SUBMISSION PREPARATION
# =============================================================================

# Create submission dataframe
submission_df = pd.DataFrame({
    "Id": test_ids_for_submission,
    "Predicted": y_pred_final.astype(int)
})

# Ensure all predictions are positive
submission_df["Predicted"] = submission_df["Predicted"].clip(lower=1)

# Display final statistics
print(f"\nFinal prediction statistics:")
print(f"Min prediction: {submission_df['Predicted'].min()}")
print(f"Max prediction: {submission_df['Predicted'].max()}")
print(f"Mean prediction: {submission_df['Predicted'].mean():.2f}")
print(f"Median prediction: {submission_df['Predicted'].median():.2f}")

print(f"\nTraining data price statistics:")
print(f"Min price: {all_train_prices.min()}")
print(f"Max price: {all_train_prices.max()}")
print(f"Mean price: {all_train_prices.mean():.2f}")
print(f"Median price: {all_train_prices.median():.2f}")

# Save submission file
submission_df.to_csv("submission.csv", index=False)
print("\nSubmission file created: submission.csv")
print("\nFirst few predictions:")
print(submission_df.head(10))

# Calculate and display feature importance
feature_importance = pd.DataFrame({
    'feature': X_train_full.columns,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

print(f"\nTop 10 most important features:")
print(feature_importance.head(10))




Final prediction statistics:
Min prediction: 371
Max prediction: 6145
Mean prediction: 1492.43
Median prediction: 1352.00

Training data price statistics:
Min price: 100.00000000000003
Max price: 5278.999999999996
Mean price: 1494.64
Median price: 1350.00

Submission file created: submission.csv

First few predictions:
   Id  Predicted
0   0        882
1   1       1936
2   2       1037
3   3       1523
4   4       1681
5   5       1604
6   6       1650
7   7       1670
8   8       1479
9   9       1725

Top 10 most important features:
                    feature  importance
0   estimated_price_by_city       31623
17                 latitude       30942
26                longitude       28865
5           log_square_feet       28271
1   cityname_target_encoded       27140
8    sqft_rooms_interaction       24763
2       city_price_per_sqft       24535
43          total_amenities       14725
56           body_tfidf_181       11681
60           title_tfidf_49       10792
